<h1>Load yours data</h1>

In [3]:
# -- Import --
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from math import *
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import sys
import ccxt
from datetime import datetime
from datetime import timedelta
from stocktrends import Renko
sys.path.append( '../../utilities' )
from data_engine import DataEngine

# -- You can change the crypto pair ,the start date and the time interval below --
pair = 'ETH/USDT'
timeframe = '1h'
startDate = '2017-01-01T00:00:00'

dataEngine = DataEngine(session=ccxt.binance(), path_to_data='../database/')

df = dataEngine.get_historical_from_db(pair, timeframe, startDate)
df['date'] = df.index

print("Data loaded 100%")

# Function to convert ohlc data into renko bricks. Pass dataframe name and brick size
def df_to_renko(data, n):
    data.reset_index(inplace=True)
    data.columns = [i.lower() for i in data.columns]
    df = Renko(data)
    df.brick_size = n
    renko_df = df.get_ohlc_data()
    return renko_df

Successfully load 40803 candles for ETHUSDT
Data loaded 100%


<h2>Trix Parameters Optimizer</h2>

In [4]:
dfTest = None
dt = None
dt = pd.DataFrame(columns = ['param1', 'result'])

# dfTest = df.copy()

# -- If you want to run your BackTest on a specific period, uncomment the line below --
dfTest = df['2022-01-01':]

def buyCondition(rowRenko):
    if rowRenko['uptrend'] == True:
        return True
    else:
        return False 

def sellCondition(rowRenko):
    if rowRenko['uptrend'] == False:
        return True
    else:
        return False

loopI = [1, 11, 1]
enumI = ceil((loopI[1] - loopI[0]) / loopI[2])

count = 0
maxCount = enumI
for i in range(loopI[0], loopI[1], loopI[2]):
  count += 1
  clear_output(wait=True)
  print("Loading...",count,'/',maxCount)
  # -- You can change variables below --
  usdt = 1000
  coin = 0

  for index, row in dfTest.iterrows():

    startRenko = datetime.strptime(str(index), '%Y-%m-%d %H:%M:%S') - timedelta(minutes=21600)
    dfComputeRenko = df[startRenko:index]
    dfRenko = df_to_renko(dfComputeRenko, row['close']*i/100)
    lastRowRenko = dfRenko.tail(1).squeeze()
    
    #BUY
    if buyCondition(lastRowRenko) and usdt > 0:
      coin = (usdt/row['close']) - 0.0007 * (usdt/row['close'])
      usdt = 0

    #SELL
    elif sellCondition(lastRowRenko) and coin > 0:
      usdt = coin * row['close'] - (0.0007 * coin * row['close'])
      coin = 0

  myrow = {'param1': i,'result': coin * dfTest.iloc[len(dfTest)-1]['close'] + usdt}
  dt = dt.append(myrow,ignore_index=True)   

dt.plot.scatter(x='param1',y=1,c='result',s=50,colormap='OrRd',figsize=(8,6))

plt.show()

print(dt.sort_values(by=['result']))

Loading... 1 / 10


KeyboardInterrupt: 